In [8]:
import numpy as np
from qiskit import QuantumCircuit, transpile


In [6]:
from math import sqrt, pow, pi
from qiskit.circuit.library import MCMT


In [7]:
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply H-gates to all qubits
    for qubit in range(nqubits):
        qc.h(qubit)
    # Implement multi-controlled-Z gate
    cnz = MCMT('z', num_ctrl_qubits=nqubits-1, num_target_qubits=1)
    qc.compose(cnz, inplace=True)
    # Apply H-gates again
    for qubit in range(nqubits):
        qc.h(qubit)
    return qc.to_gate()

# Define the string size
n = 6

# Create the oracle
qc_oracle = QuantumCircuit(n+1)
qc_oracle.cx(1, n)
qc_oracle.cx(0, n)  # String is equal to 110000
oracle_gate = qc_oracle.to_gate()
oracle_gate.name = "U_omega"

# Create the Grover circuit
grover_circuit = QuantumCircuit(n+1, n)
grover_circuit.x(n)  # Create the Ancilla |->

# Apply Hadamard gates to all qubits
grover_circuit.h(range(n+1))

# Number of iterations of oracle and diffuser needed
t = int(pi * sqrt(pow(pow(2, n), 0.5)) / 4 - 0.5)

for _ in range(t):
    grover_circuit.append(oracle_gate, range(n+1))
    grover_circuit.append(diffuser(n), range(n))

# Apply Hadamard gates again
grover_circuit.h(range(n))

# Measure all qubits
grover_circuit.measure(range(n), range(n))

# Draw the circuit
grover_circuit.draw()

┌───┐     ┌──────────┐┌──────────────┐┌───┐┌─┐               
q_0: ┤ H ├─────┤0         ├┤0             ├┤ H ├┤M├───────────────
     ├───┤     │          ││              │├───┤└╥┘┌─┐            
q_1: ┤ H ├─────┤1         ├┤1             ├┤ H ├─╫─┤M├────────────
     ├───┤     │          ││              │├───┤ ║ └╥┘┌─┐         
q_2: ┤ H ├─────┤2         ├┤2             ├┤ H ├─╫──╫─┤M├─────────
     ├───┤     │          ││  circuit-171 │├───┤ ║  ║ └╥┘┌─┐      
q_3: ┤ H ├─────┤3 U_omega ├┤3             ├┤ H ├─╫──╫──╫─┤M├──────
     ├───┤     │          ││              │├───┤ ║  ║  ║ └╥┘┌─┐   
q_4: ┤ H ├─────┤4         ├┤4             ├┤ H ├─╫──╫──╫──╫─┤M├───
     ├───┤     │          ││              │├───┤ ║  ║  ║  ║ └╥┘┌─┐
q_5: ┤ H ├─────┤5         ├┤5             ├┤ H ├─╫──╫──╫──╫──╫─┤M├
     ├───┤┌───┐│          │└──────────────┘└───┘ ║  ║  ║  ║  ║ └╥┘
q_6: ┤ X ├┤ H ├┤6         ├──────────────────────╫──╫──╫──╫──╫──╫─
     └───┘└───┘└──────────┘                      ║  ║  ║  ║  ║  ║ 
c: 6/════════════════════════════════════════════╩══╩══╩══╩══╩══╩═
                                                 0  1  2  3  4  5